<a href="https://colab.research.google.com/github/Dr-Carlos-Villasenor/Clase_Aprendizaje_Profundo/blob/main/L11_02_LSTM_nombres_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aprendizaje profundo
##Dr. Carlos Villaseñor
### Practica de generación de nombres con LSTM


Primeramente importamos las paqueterias necesarias

In [ ]:
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils

Vamos a abrir el archivo con conmbres y limpar un poco los datos

In [ ]:
text = open('names.txt').read()
text = text.lower()
names = text.split('\n')
names = list(filter(lambda name: ' ' not in name, names))
names = list(filter(lambda name: '-' not in name, names))
names = list(filter(lambda name: "'" not in name, names))
names = [name + '.' for name in names]
names = np.random.permutation(names)
print(names)

Vamos a crear un diccionario que con el cual podamos mapear de caracteres a números

In [ ]:
char_to_index = dict( (chr(i+96), i) for i in range(1,27))
char_to_index[' '] = 0
char_to_index['.'] = 27
print(char_to_index)

También necesitaremos el mapeo inverso

In [ ]:
index_to_char = dict( (i, chr(i+96)) for i in range(1,27))
index_to_char[0] = ' '
index_to_char[27] = '.'
print(index_to_char)

Calculemos el número de nombres y el tamaño máximo que un nombre puede tener

In [ ]:
num_names = len(names)
max_name_size = len(max(names, key=len))
num_chars = len(char_to_index)
print(max_name_size)

Vamos a códificar todos los nombres y a crear un dataset con los nombres y su retardo

In [ ]:
x = np.zeros((num_names, max_name_size, num_chars))
y = np.zeros((num_names, max_name_size, num_chars))
print(x.shape)

for i, name in enumerate(names):
  for j, c in enumerate(name):
    x[i, j, char_to_index[name[j]]] = 1
    if j < len(name)-1:
      y[i, j, char_to_index[name[j+1]]] = 1

Intanciamos nuestra red recurrente LSTM

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(max_name_size, num_chars),return_sequences=True))
model.add(Dropout(0.25))
model.add(Dense(num_chars, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['acc'])
model.summary()

Momento de entrenar!!

In [ ]:
model.fit(x, y, batch_size=512, epochs=200)

El modelo está entrenado, pero ¿Cómo generamos nombres?!!

Para esto está el código siguiente!!


In [ ]:
# Correr un ejemplo
name = ''
x = np.zeros((1,max_name_size, num_chars))
first = True
end = False
i = 0
while end==False:
  if first == True:
    probs = list(model.predict(x, verbose=0)[0,i])[1:-1]
    probs = probs/ np.sum(probs)
    index = np.random.choice(range(1,num_chars-1),p=probs)
    first = False
  else:
    probs = list(model.predict(x, verbose=0)[0,i])
    probs = probs/ np.sum(probs)
    index = np.random.choice(range(num_chars),p=probs)
  if i == max_name_size-1:
    break
    character = '.'
    end = True
  else:
    character = index_to_char[index]
  name += character
  x[0, i+1, index] = 1
  i += 1
  if character == '.':
    end = True

print(name.capitalize())
print('Memorized: ', name in names)